# NLP With Deep Learning (W266)

Submission by *Carolina Arriaga, Ayman, Abhi Sharma*

Winter 2021 | UC Berkeley

# Utils

In [ ]:
import datetime
from google.colab import files

def write_to_csv(df, inner_filename):
  now = datetime.datetime.now()
  filename = now.strftime("%Y-%m-%d-%H-%M-%S")

  compression_opts = dict(method='zip', archive_name='{}.csv'.format(inner_filename))

  df.to_csv('{}.zip'.format(filename), index=False, compression = compression_opts)
  files.download('{}.zip'.format(filename))

# Data Prep

In [ ]:
import pandas as pd

# need upload file (drive/data)
scored = pd.read_csv('/content/data_scored_with_top_tranmatrix.csv')
scored.rename(columns={'story_id':'id'}, inplace=True)

summeval = pd.read_csv('/content/summeval_scores.csv')
summeval = summeval[summeval['model_id'] != 'M23_dynamicmix']

In [ ]:
merged = pd.merge(scored, summeval, on=['id', 'model_id'])

In [ ]:
len(merged)

1653

In [ ]:
merged.rename(columns={'decoded_x':'decoded'}, inplace=True)
merged = merged.drop(['decoded_y'], axis=1)

In [ ]:
pearson = merged.corr(method ='pearson')
kendall = merged.corr(method ='kendall')

In [ ]:
write_to_csv(pearson, 'pearson')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# sleep to avoid zip filenames being the same
!sleep 5

In [ ]:
write_to_csv(kendall, 'kendall')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pearson.head()

,expert_0_coherence,expert_1_coherence,expert_2_coherence,all_expert_avg_coherence,expert_0_consistency,expert_1_consistency,expert_2_consistency,all_expert_avg_consistency,expert_0_fluency,expert_1_fluency,expert_2_fluency,all_expert_avg_fluency,expert_0_relevance,expert_1_relevance,expert_2_relevance,all_expert_avg_relevance,turk_0_coherence,turk_1_coherence,turk_2_coherence,turk_3_coherence,turk_4_coherence,all_turk_avg_coherence,turk_0_consistency,turk_1_consistency,turk_2_consistency,turk_3_consistency,turk_4_consistency,all_turk_avg_consistency,turk_0_fluency,turk_1_fluency,turk_2_fluency,turk_3_fluency,turk_4_fluency,all_turk_avg_fluency,turk_0_relevance,turk_1_relevance,turk_2_relevance,turk_3_relevance,turk_4_relevance,all_turk_avg_relevance,...,metric_scores_11_rouge_rouge_l_f_score_cb,metric_scores_11_rouge_rouge_l_f_score_ce,metric_scores_11_rouge_rouge_w_1.2_recall,metric_scores_11_rouge_rouge_w_1.2_recall_cb,metric_scores_11_rouge_rouge_w_1.2_recall_ce,metric_scores_11_rouge_rouge_w_1.2_precision,metric_scores_11_rouge_rouge_w_1.2_precision_cb,metric_scores_11_rouge_rouge_w_1.2_precision_ce,metric_scores_11_rouge_rouge_w_1.2_f_score,metric_scores_11_rouge_rouge_w_1.2_f_score_cb,metric_scores_11_rouge_rouge_w_1.2_f_score_ce,metric_scores_11_rouge_rouge_s*_recall,metric_scores_11_rouge_rouge_s*_recall_cb,metric_scores_11_rouge_rouge_s*_recall_ce,metric_scores_11_rouge_rouge_s*_precision,metric_scores_11_rouge_rouge_s*_precision_cb,metric_scores_11_rouge_rouge_s*_precision_ce,metric_scores_11_rouge_rouge_s*_f_score,metric_scores_11_rouge_rouge_s*_f_score_cb,metric_scores_11_rouge_rouge_s*_f_score_ce,metric_scores_11_rouge_rouge_su*_recall,metric_scores_11_rouge_rouge_su*_recall_cb,metric_scores_11_rouge_rouge_su*_recall_ce,metric_scores_11_rouge_rouge_su*_precision,metric_scores_11_rouge_rouge_su*_precision_cb,metric_scores_11_rouge_rouge_su*_precision_ce,metric_scores_11_rouge_rouge_su*_f_score,metric_scores_11_rouge_rouge_su*_f_score_cb,metric_scores_11_rouge_rouge_su*_f_score_ce,metric_scores_11_bleu,metric_scores_11_rouge_we_2_p,metric_scores_11_rouge_we_2_r,metric_scores_11_rouge_we_2_f,metric_scores_11_meteor,metric_scores_11_rouge_we_1_p,metric_scores_11_rouge_we_1_r,metric_scores_11_rouge_we_1_f,metric_scores_11_mover_score,metric_scores_11_chrf,metric_scores_11_sentence_movers_glove_sms
expert_0_coherence,1.000000,0.725007,0.616291,0.889606,0.305461,0.306979,0.285649,0.308912,0.379442,0.307915,0.290440,0.357461,0.620446,0.516509,0.391879,0.624515,0.002904,0.037274,0.007769,0.036223,0.060284,0.037486,0.013703,0.053094,0.011226,0.019444,0.058665,0.040350,0.033791,0.068244,0.014282,0.072392,0.061312,0.065267,0.025953,0.034654,0.045322,0.035479,0.061629,0.053144,...,0.214842,0.214842,0.239467,0.239467,0.239467,0.137972,0.137972,0.137972,0.225116,0.225116,0.225116,0.275948,0.275948,0.275948,0.196553,0.196553,0.196553,0.250156,0.250156,0.250156,0.274478,0.274478,0.274478,0.184557,0.184557,0.184557,0.243780,0.243780,0.243780,0.221943,0.186814,0.250854,0.223599,0.228133,0.175576,0.270742,0.255573,0.226457,0.262618,0.241555
expert_1_coherence,0.725007,1.000000,0.616804,0.916812,0.281753,0.303992,0.269229,0.294832,0.367628,0.315002,0.303581,0.360779,0.496600,0.582615,0.359471,0.592021,-0.003152,0.017630,0.022839,0.031076,0.070595,0.036132,-0.009691,0.054307,0.026367,0.037597,0.094382,0.052242,0.025620,0.060791,0.034930,0.057128,0.078442,0.067121,-0.003649,0.027466,0.039867,0.029662,0.071162,0.042859,...,0.104391,0.104391,0.074725,0.074725,0.074725,0.128704,0.128704,0.128704,0.086132,0.086132,0.086132,0.126307,0.126307,0.126307,0.186992,0.186992,0.186992,0.145033,0.145033,0.145033,0.124259,0.124259,0.124259,0.183487,0.183487,0.183487,0.142778,0.142778,0.142778,0.139839,0.127686,0.085454,0.095783,0.101954,0.175514,0.118540,0.154546,0.186020,0.181893,0.165371
expert_2_coherence,0.616291,0.616804,1.000000,0.817542,0.218139,0.231865,0.215971,0.229278,0.308794,0.236690,0.250248,0.289972,0.412733,0.423003,0.458647,0.521100,

# Column Wise Analysis

In [ ]:
def get_corr(df, col1, col2, verbose=True):
  cor = df[col1].corr(df[col2])
  if verbose:
    print ("Correlation between {} and {} is {}".format(col1, col2, cor))
  return col1, col2, cor

def get_corrs(df, fixed_col):
  cols = list(df.columns)
  res = []
  for c in cols:
    # cant do correlations against string
    if isinstance(df[c][0], str) or isinstance(df[fixed_col][0], str):
      continue

    col1, col2, cor = get_corr(df, c, fixed_col, verbose=False)
    res.append({'col1': col1, 'col2': col2, 'pearson_corr': cor})
  return res    

def get_corrs_threshold(df, fixed_col, threshold=0.5, above=True):
  res = get_corrs(df, fixed_col)
  if above:
    res = [r for r in res if r['pearson_corr'] >= threshold]
  else:
    res = [r for r in res if r['pearson_corr'] <= threshold]
  return res

def get_expert_turk_cols():
  expert_turk_cols = [col for col in list(merged.columns) if 'expert' in col or 'turk' in col]
  expert_turk_cols = [col for col in expert_turk_cols if 'annotation' not in col]
  return expert_turk_cols

def get_non_expert_turk_cols():
  expert_turk_cols = [col for col in list(merged.columns) if 'expert' in col or 'turk' in col]
  return [c for c in list(merged.columns) if c not in expert_turk_cols]

Sanity Check

In [ ]:
get_corr(merged, "turker_annotations_2_coherence", "turk_2_coherence")
get_corr(merged, "turker_annotations_3_fluency", "turk_3_fluency")
get_corr(merged, "expert_annotations_0_consistency", "expert_0_consistency")
get_corr(merged, "expert_annotations_1_relevance", "expert_1_relevance")

Correlation between turker_annotations_2_coherence and turk_2_coherence is 1.0
Correlation between turker_annotations_3_fluency and turk_3_fluency is 1.0
Correlation between expert_annotations_0_consistency and expert_0_consistency is 1.0
Correlation between expert_annotations_1_relevance and expert_1_relevance is 1.0


('expert_annotations_1_relevance', 'expert_1_relevance', 1.0)

In [ ]:
get_corrs_threshold(merged, 'expert_annotations_0_consistency', threshold=0.7, above=True)

[{'col1': 'expert_0_consistency',
  'col2': 'expert_annotations_0_consistency',
  'pearson_corr': 1.0},
 {'col1': 'expert_1_consistency',
  'col2': 'expert_annotations_0_consistency',
  'pearson_corr': 0.893419055244002},
 {'col1': 'expert_2_consistency',
  'col2': 'expert_annotations_0_consistency',
  'pearson_corr': 0.9055273497575658},
 {'col1': 'all_expert_avg_consistency',
  'col2': 'expert_annotations_0_consistency',
  'pearson_corr': 0.9586900011831876},
 {'col1': 'expert_annotations_0_consistency',
  'col2': 'expert_annotations_0_consistency',
  'pearson_corr': 1.0},
 {'col1': 'expert_annotations_1_consistency',
  'col2': 'expert_annotations_0_consistency',
  'pearson_corr': 0.893419055244002},
 {'col1': 'expert_annotations_2_consistency',
  'col2': 'expert_annotations_0_consistency',
  'pearson_corr': 0.9055273497575658}]

Iterate over expert and turker scores to see if any of them correlate with a real feature

In [ ]:
aggregated_corrs_et = []
et_cols = get_expert_turk_cols()
for c in et_cols:
  aggregated_corrs_et.extend(get_corrs_threshold(merged, c, threshold=0.25))

# filter self correlations
aggregated_corrs_et = [cor for cor in aggregated_corrs_et if 'expert' not in cor['col1'] and 'turk' not in cor['col1']]
# sort from highest corr to lowest
aggregated_corrs_et = sorted(aggregated_corrs_et, key=lambda x: x['pearson_corr'], reverse=True)

In [ ]:
aggregated_corrs_et

[{'col1': 'summary_stats_coverage',
  'col2': 'expert_2_consistency',
  'pearson_corr': 0.41659193465605016},
 {'col1': 'summary_stats_coverage',
  'col2': 'all_expert_avg_consistency',
  'pearson_corr': 0.41319701084427785},
 {'col1': 'summary_stats_coverage',
  'col2': 'expert_1_consistency',
  'pearson_corr': 0.40023734057728205},
 {'col1': 'metric_scores_11_rouge_rouge_1_recall',
  'col2': 'all_expert_avg_relevance',
  'pearson_corr': 0.38998692338703556},
 {'col1': 'metric_scores_11_rouge_rouge_1_recall_cb',
  'col2': 'all_expert_avg_relevance',
  'pearson_corr': 0.38998692338703556},
 {'col1': 'metric_scores_11_rouge_rouge_1_recall_ce',
  'col2': 'all_expert_avg_relevance',
  'pearson_corr': 0.38998692338703556},
 {'col1': 'metric_scores_11_rouge_we_1_r',
  'col2': 'all_expert_avg_relevance',
  'pearson_corr': 0.3887521245913678},
 {'col1': 'metric_scores_11_rouge_rouge_1_recall',
  'col2': 'expert_1_relevance',
  'pearson_corr': 0.3877462989905209},
 {'col1': 'metric_scores_11_r

Do within feature correlations to see if features correlate with each other

In [ ]:
aggregated_corrs_features = []
feature_cols = get_non_expert_turk_cols()
for c in feature_cols:
  aggregated_corrs_features.extend(get_corrs_threshold(merged, c, threshold=0.75))

# filter self correlations
aggregated_corrs_features = [cor for cor in aggregated_corrs_features if cor['col1'] != cor['col2']]
# sort from highest corr to lowest
aggregated_corrs_features = sorted(aggregated_corrs_features, key=lambda x: x['pearson_corr'], reverse=True)

In [ ]:
aggregated_corrs_features[-5:]

[{'col1': 'metric_scores_11_rouge_we_2_r',
  'col2': 'metric_scores_11_rouge_rouge_4_recall_cb',
  'pearson_corr': 0.7502138685067093},
 {'col1': 'metric_scores_11_rouge_we_2_r',
  'col2': 'metric_scores_11_rouge_rouge_4_recall_ce',
  'pearson_corr': 0.7502138685067093},
 {'col1': 'metric_scores_11_rouge_rouge_4_recall',
  'col2': 'metric_scores_11_rouge_we_2_r',
  'pearson_corr': 0.7502138685067093},
 {'col1': 'metric_scores_11_rouge_rouge_4_recall_cb',
  'col2': 'metric_scores_11_rouge_we_2_r',
  'pearson_corr': 0.7502138685067093},
 {'col1': 'metric_scores_11_rouge_rouge_4_recall_ce',
  'col2': 'metric_scores_11_rouge_we_2_r',
  'pearson_corr': 0.7502138685067093}]

In [ ]:
# the list of "our" metrics that have corr >= 0.25
set([(c['col1']) for c in aggregated_corrs_et if 'metric_scores_11' not in c['col1']])

{'avg_slor_sents_decoded',
 'bert_score_recall',
 'meteor_metric',
 'rouge_1_mid_recall',
 'rouge_2_mid_recall',
 'rouge_4_skipgram_recall',
 'rouge_L_mid_recall',
 'rouge_Lsum_mid_recall',
 'rouge_recall_max_sents',
 'rouge_recall_mean_sents',
 'rouge_recall_mean_summ',
 'rouge_recall_median_sents',
 'rouge_recall_min_sents',
 'rouge_we_1_f1',
 'rouge_we_1_recall',
 'rouge_we_2_f1',
 'rouge_we_2_recall',
 'rouge_we_3_f1',
 'rouge_we_3_recall',
 'rouge_we_4_f1',
 'rouge_we_4_recall',
 'rouge_wt_sum30_recall',
 'rouge_wt_sum50_recall',
 'rouge_wt_sum70_recall',
 'summary_stats_coverage',
 'summary_stats_density'}

In [ ]:
# interesting that the bert scores are not correlated perfectly
get_corr(merged, 'metric_scores_11_bert_score_recall', 'bert_score_recall')

Correlation between metric_scores_11_bert_score_recall and bert_score_recall is 0.5883321732358494


('metric_scores_11_bert_score_recall', 'bert_score_recall', 0.5883321732358494)

In [ ]:
# same with chrf
get_corr(merged, 'metric_scores_11_chrf', 'chrf_metric')

Correlation between metric_scores_11_chrf and chrf_metric is -0.2573797256607761


('metric_scores_11_chrf', 'chrf_metric', -0.2573797256607761)

In [ ]:
get_corr(merged, 'metric_scores_11_meteor', 'meteor_metric')

Correlation between metric_scores_11_meteor and meteor_metric is 0.6036474777636461


('metric_scores_11_meteor', 'meteor_metric', 0.6036474777636461)